# 'Segmentación' de una imagen en el espacio de color

La aproximación que hemos usado hasta ahora para presentar DBSCAN se ha basado en ejemplos donde el concepto de 'localidad' venia dado por la contiguidad espacial. En este ejercicio vamos a usar DBSCAN en el espacio de características del color y vamos a usar DBSCAN para agrupar píxeles de una imágen que tienen color parecido (dependerá del parámetro Epsilon que elijas).

La segmentación de una imagen en el campo de la visión por computador, consiste en la identificación de objetos diferentes en la misma. Formalmente esto se hace generalmente usando operaciones sobre pixel (detección de bordes, regiones,...). Nosotros vamos a utilizar DBSCAN para agrupar en el espacio de colores de la imagen  los pixeles que tengan colores muy parecidos. No es una verdadera segmentación, pero es una aproximación al problema de la detección de regiones de la imagen que representan objetos diferentes (dejando a un lado el problema de las sombras, reflejos etc)

### Paso 1. Carga de la imagen
Carga la imagen 'mercado.jpg' utilizando la libreria de Python cv2 y escálala al 50% ya que es un poco grande para el procesamiento de DBSCAN.Para ello usa cv2.resize().
Comprueba quie todo ha ido bien escribiendo la imagen escalada usando cv2.imwrite() y visualizándola a su vez con Image(filename=??)
La imagen se carga en RGB, cada pixel tiene 3 valores en un rango [0,255] que representa su color.

In [ ]:
import cv2 #EDIT, this line added
from IPython.display import Image

import numpy as np
from sklearn.cluster import DBSCAN

src = cv2.imread('mercado.jpg', cv2.IMREAD_UNCHANGED)
 
#Porcentaje en el que se redimensiona la imagen
scale_percent = 50
 
#calcular el 50 por ciento de las dimensiones originales usando los datos de src.shape
width = int(???)
height = int(???)
 
# dsize
dsize = (width, height)
 
# cambiar el tamaño de la imagen
output = cv2.resize(???)
#output = src
 
cv2.imwrite('mercado_half.jpg',output) 

Image(filename='mercado_half.jpg') 

### Paso 2 Cambio de formato de la imagen
La imagen esta en memoria como una matriz de pixeles de 3 componentes, pero DBSCAN procesa vectores de datos, así que vamos a cambiar el formato por un vector de componentes de color. Es decir vamos a cambiar de una matriz de dimensiones (n,m,3) a un vector de dimensiones (n*m,3)

In [ ]:
#Comprobamos la talla de nuestra imagen
print(output.shape)
sh = output.shape
#print(output)
#print(sh[0],sh[1])

#Reformateamos la imagen 'output' usando np.reshape a (sh[0]*sh[1],3)
output_std = ???
print("*******")

#debe dar (70794,3)
print(output_std.shape)

### Paso 3. Aplicamos DBSCAN
Como sabemos, DBSCAN es muy sensible a los parámetros **eps y MinPts**. Una variación de 1 unidad en 'eps' puede representar un cambio de un resultado aceptable a un mal resultado o viceversa. Estudia el resultado de DBSCAN para un rango **eps [2,6]** aumentando de 1 en 1 y para un **MinPts [10, 20]**  aumentando de 1 en 1. Como no puedes comparar con una solucion, una forma de ver si el resultado puede ser bueno es observar el número de etiquetas que genera DBSCAN (número de clusters) y la cantidad de píxeles que  contiene cada cluster (tamaño).
Como indicación en nuestra experiencia con esta imagen, una buena indicación es:
-obtener más de 100 clusters
-el cluster etiquetado como -1 (ruido) tiene menos del 50% de los datos

In [ ]:
# Define max_distance (eps parameter in DBSCAN())
max_distance = ??
db = DBSCAN(eps=max_distance, min_samples=??).fit(output_std)

# Extract labels (-1 is used for outliers)
labels = ???

Con la información que te proporciona la caja siguiente, puedes juzgar si el resultado será bueno atendiendo al número de grupos formados y a la cantidad de pixeles que tiene cada grupo. No hay un resultado claramente mejor aunque sí muchos mejores que otros. Piensa que estás clusterizando por  color de pixel, no por contiguidad espacial de pixels.

In [ ]:
#Analizamos el número de clusters y su tamaño
print(db.n_features_in_)
unique, counts = np.unique(labels, return_counts=True)
print(unique)
print(counts)

Preparamos ahora una paleta de colores para pintar cada pixel con un color según su pertenencia a un cluster.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

lst = unique

fig = plt.figure(figsize=(7,5))
axes = fig.add_subplot(1,1,1)
axes.set_ylim(0, len(lst))
plt.style.use("seaborn")


lst = [x+2 for x in lst]
palette = sns.color_palette("Spectral",len(lst))
#palette = sns.color_palette("inferno_r", 7)

print(palette)
plt.bar(range(len(lst)), sorted(lst), color=palette)
#tick_lst=["one", "two", "three", "four", "five", "six", "seven"]
plt.xticks(np.arange(len(lst)))
plt.title("Reversed Inferno Palette 7 Colors", color="darkblue")

EL código de la caja siguiente pretende evitar que clusters muy grandes tengan colores muy parecidos.
Observando el vector de clusters con su tamaño, que has calculado anteriormente, podrás  darte cuenta de que muchas veces aparecen clusteres muy grandes juntos al principio (tipicamente el cluster de etiqueta -1 que corresponde al ruido y algun cluster entre la etiqueta 0 y 10). Como las etiquetas tambien representan entradas a la paleta de colores, etiquetas muy próximas tambien tienen colores muy proximos.
Esto hace que sea difícil identificar los diferentes grupos grandes. Para evitar esto, vamos a intercambiar las etiquetas de al menos el grupo de ruido y otro grupo que consideres cercano al mismo, intercambiando las etiquetas con otros grupos con etiquetas del final (ej. intercambiamos las etiquetas -1 con otra etiqueta del final que tenga pocos pixeles) e intercambiamos el otro grupo grande cercano a -1 con una etiqueta de un grupo pequeño del medio del vector de etiquetas.

In [ ]:
#Intercambio de algunas etiquetas por parte de los pixeles
for j in range(0,sh[0]*sh[1]):
    aux = labels[j]
    
    #elijo un grupo próximo a la etiqueta '-1' que sea muy grande y lo mando a otra region de color
    if labels[j] == 0:
        aux = 35
    if labels[j] == 35:
        aux = 0
    #elijo la etiqueta de ruido (-1) y la intercambio con otra del final de la paleta
    if labels[j] == -1:
        aux = 75
    if labels[j] == 75:
        aux = -1
    
    labels[j] = aux

Ahora reescribimos los colores de los pixeles de la imagen con los colores de la paleta siguiendo los valores de la etiquetas (es decir, pintamos cada pixel con el color asociado a la etiqueta de dicho pixel)

In [ ]:

for i in range(0,sh[0]*sh[1]): 
    output_std[i,0] = round((np.array(palette[labels[i]+1])[0]) * 255.0)
    output_std[i,1] = round((np.array(palette[labels[i]+1])[1]) * 255.0)
    output_std[i,2] = round((np.array(palette[labels[i]+1])[2]) * 255.0)


Reescalamos el vector de píxeles para que vuelva a ser una matriz de pixeles (formato original de la imagen).
Y escribimos la imagen procesada en un fichero para su visualizacion.

In [ ]:
output_newshape = np.resize(???)
print(output_newshape.shape)

cv2.imwrite('mercado_segmentado.jpg',cv2.cvtColor(output_newshape, cv2.COLOR_RGB2BGR)) 


Image(filename='mercado_segmentado.jpg') 

**Responde a las siguientes preguntas: (dando una explicación razonable)**
P1: Indica tu experiencia con el parámetro Epsilon a partir de tus resultados
P2: ¿Es/Sería posible que un pixel de pimiento y un pixel de un tomate pertenezcan al mismo cluster?